In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import GridSearchCV, train_test_split
import joblib
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv('prob.csv')

In [ ]:
x = df[['T', 'B', 'C1', 'C2', 'C3', 'M1', 'M2', 'M3']]
y = df['Improved_Churn_Probability']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

print(x_train)
model = xgb.XGBRegressor(objective='reg:squarederror', verbosity=0)  


# searching best hyperparams
param_grid = {
    'max_depth': [6, 10, 12],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'reg_lambda': [1, 3, 5],
    'booster': ['gbtree', 'dart']
}

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,  # cross validation
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(x_train, y_train)

# best params
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

# train with best params
best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror', verbosity=0)
best_model.fit(x_train, y_train)

best_model.save_model('best_xgboost_model.json') 
